In [1]:
from jaxodi.doppler_surface import DopplerWav, DopplerSurface

In [2]:
from jaxoplanet.experimental.starry.ylm import Ylm

In [3]:
import numpy as np

In [4]:
y = Ylm.from_dense(np.hstack([1.0, np.random.rand(35) * 0.3]))

In [6]:
y

Ylm(
  data={
    (0, 0):
    1.0,
    (1, -1):
    0.23505291558187855,
    (1, 0):
    0.16975382701116942,
    (1, 1):
    0.0352616000908861,
    (2, -2):
    0.05452717714613513,
    (2, -1):
    0.08992025777970514,
    (2, 0):
    0.16685579061380604,
    (2, 1):
    0.015836030898152664,
    (2, 2):
    0.1620923330819826,
    (3, -3):
    0.13817522941862473,
    (3, -2):
    0.1232808460354058,
    (3, -1):
    0.12062883323032463,
    (3, 0):
    0.2280140752780554,
    (3, 1):
    0.10453151367709494,
    (3, 2):
    0.2860131863289307,
    (3, 3):
    0.005730641480622722,
    (4, -4):
    0.2834841625511442,
    (4, -3):
    0.23728559812973607,
    (4, -2):
    0.18022486179562439,
    (4, -1):
    0.0354277296829934,
    (4, 0):
    0.20230058811414225,
    (4, 1):
    0.18297401592983123,
    (4, 2):
    0.003855222935747171,
    (4, 3):
    0.057487387011595115,
    (4, 4):
    0.0382191909992811,
    (5, -5):
    0.21182739888349386,
    (5, -4):
    0.21178453358497

In [7]:
y.ell_max

5

In [8]:
wav = DopplerWav(vsini_max=60000.)

In [9]:
wav

DopplerWav(wav=f64[200](numpy), vsini_max=60000.0)

In [10]:
spec0 = 1.0 - np.exp(-0.5 * (wav.wav0 - 643.0) ** 2 / 0.0085 ** 2)

In [11]:
spec0.shape

(304,)

In [12]:
theta = np.append([-np.pi], np.linspace(-np.pi/2, np.pi/2, 15))

In [13]:
dm = DopplerSurface(theta)

In [14]:
dm

DopplerSurface(theta=f64[16](numpy), nt=16, inc=1.5707963267948966, obl=0)

In [15]:
flux = dm(y, wav, spec0)

In [16]:
flux.shape

(16, 201)